In [ ]:
from pathlib import Path
import pyreadr
import pandas as pd

# 🔧 Caminho base onde estão as subpastas com datas
base_dir = Path(r"G:\My Drive\Github\r-2023-caixaleiloes-new\csv Caixa\resultados_licitacao_aberta")

# 📂 Pasta onde o Jupyter Notebook está rodando
output_dir = Path.cwd()

print("Notebook directory:", output_dir)

all_dfs = []

# 🔍 Varrer todas as subpastas e todos os arquivos .Rda
for rda_path in base_dir.rglob("*.Rda"):
    print(f"Lendo: {rda_path}")
    result = pyreadr.read_r(str(rda_path))

    # result = dict {nome_objeto: dataframe}
    for name, obj in result.items():

        # ❌ Ignorar df_all
        if name == "df_all":
            continue

        # ✔️ Aceitar só DataFrame
        if isinstance(obj, pd.DataFrame):
            obj = obj.copy()
            obj["source_folder"] = rda_path.parent.name
            obj["source_file"] = rda_path.name
            obj["source_obj"] = name
            all_dfs.append(obj)

# ➕ Merge final
if all_dfs:
    df_merged = pd.concat(all_dfs, ignore_index=True)
    print("Linhas totais:", len(df_merged))

    # ✔️ Caminho final do arquivo parquet
    output_path = output_dir / "resultados_licitacao_aberta_merged.parquet"

    # 💾 Salvar parquet
    df_merged.to_parquet(output_path, index=False)

    print(f"Arquivo salvo em:\n{output_path}")

else:
    print("Nenhum dataframe encontrado nas pastas (exceto df_all).")
